### Pull Author Data 
For each DD post, pull data for authors that is included in the bot-generated post. 
- Earliest WSB appearance 
- Total comments up until post date time 
- Total posts up intil post date time
- Links to previous DD posts - pull post ID for previous DD posts 

For each DD post author, pull all previous posts (including flair to identify DD posts) and all previous comments 

In [1]:
# load packages
import praw 
import pandas as pd 
import datetime as dt 
from datetime import timedelta
import numpy as np 
from psaw import PushshiftAPI
import re

#### Set up 
1. Initialize instance of Pushshift API wrapper
2. Load dd post data 

In [16]:
# load reddit and pushshift API for accessing posts
reddit = praw.Reddit(client_id = 'rAfwBiShkge-tw',
                    client_secret = '1rcUS_cY1tZ_uDGwJdpEIe-vSow',
                    user_agent = 'wsb_app',
                    username = 'Real_Measurement_493', 
                    password = 'stella96')
api = PushshiftAPI(reddit)

In [5]:
dd_df = pd.read_pickle('/Volumes/Elements/Research/Reddit_Credibility/Data/dd_df_sample.pkl')

### Pull all submissions made by dd post authors

In [4]:
# get list of unique authors 
authors = dd_df.author.unique()
authors = list(filter(None, authors))

In [36]:
# initialize post list and comment list 
author_post_list = []
author_comment_list = []

In [25]:
# loop through authors 
for i in range(0, len(authors)):
    
    a = authors[i]
    
    # print counter 
    if i % 500 == 0:
        print('============================')
        print('On submission ' + str(i) + ' out of ' + str(len(authors)))
    
    # search for all submissions 
    sub_search = api.search_submissions(subreddit = 'wallstreetbets', author = a, 
                                       filter = ['url', 'author', 'title', 'body', 'created', 
                                                'num_comments', 'score', 'link_flair_text'])
    # loop through sub_search 
    for submission in sub_search:
        
        # if submission hasn't been removed
        if submission.is_robot_indexable:
        
            # pull all needed data
            title = submission.title
            author = submission.author
            score = submission.score
            submission_id = submission.id
            url = submission.url
            comms_num = submission.num_comments
            created = submission.created
            flair = submission.link_flair_text 
            body = submission.selftext

            # append to author post list 
            author_post_list.append((a, title, author, score, submission_id, url, 
                                    comms_num, created, flair, body))
        

On submission 0 out of 6258


/Users/kimmiegeorge/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/Users/kimmiegeorge/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


On submission 500 out of 6258
On submission 1000 out of 6258
On submission 1500 out of 6258
On submission 2000 out of 6258
On submission 2500 out of 6258
On submission 3000 out of 6258
On submission 3500 out of 6258
On submission 4000 out of 6258
On submission 4500 out of 6258
On submission 5000 out of 6258
On submission 5500 out of 6258
On submission 6000 out of 6258


In [26]:
# convert list to df 
author_post_df = pd.DataFrame(author_post_list, columns = ['author', 'title', 'author2', 'score', 
                                                          'submission_id', 'url', 'comms_num', 'created', 'flair', 'body'])


# remove duplicate posts (from re-starting loop)
author_post_df = author_post_df.drop_duplicates()

In [29]:
# save author post df 
author_post_df.to_pickle('/Volumes/Elements/Research/Reddit_Credibility/Data/posts_by_dd_authors.pkl')

In [11]:
# run search for Kremfloete - did not work 
dd_df['author_str'] = dd_df['author'].astype(str)

In [14]:
auth = dd_df[dd_df['author_str'] == 'Kremfloete'].author

In [17]:
# search for all submissions 
sub_search = api.search_submissions(subreddit = 'wallstreetbets', author = auth, 
                                       filter = ['url', 'author', 'title', 'body', 'created', 
                                                'num_comments', 'score', 'link_flair_text'])

In [40]:
del author_post_list

In [41]:
del author_post_df

### Pull all comments made by dd post authors

In [42]:
# loop through authors
for i in range(421, len(authors)):
    
   
    a = authors[i]
    
    # print counter 
    if i % 500 == 0:
        print('============================')
        print('On author ' + str(i) + ' out of ' + str(len(authors)))
    
    # search for all submissions 
    comm_search = api.search_comments(subreddit = 'wallstreetbets', author = a, 
                                       filter = ['id','body', 'author','parent_id', 'score', 'created'])
    # loop through sub_search 
    for comment in comm_search:
        
        # if comment body not equal to removed 
        if comment.body != '[removed]':
        
            # pull all needed data
            _id = comment.id
            author = comment.author
            body = comment.body
            parent_id = comment.parent_id
            score = comment.score
            created = comment.created

            # append to author post list 
            author_comment_list.append((a, _id, author, body, parent_id, score, created))
    

On author 500 out of 6258
On author 1000 out of 6258
On author 1500 out of 6258
On author 2000 out of 6258
On author 2500 out of 6258


/Users/kimmiegeorge/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


On author 3000 out of 6258
On author 3500 out of 6258
On author 4000 out of 6258
On author 4500 out of 6258
On author 5000 out of 6258
On author 5500 out of 6258
On author 6000 out of 6258


In [43]:
# convert list to df 
author_comment_df = pd.DataFrame(author_comment_list, columns = ['author', 'comment_id', 'author2', 'body', 
                                                          'parent_id', 'score', 'created'])


# remove duplicate posts (from re-starting loop)
author_comment_df = author_comment_df.drop_duplicates()

In [46]:
author_comment_df.to_pickle('/Volumes/Elements/Research/Reddit_Credibility/Data/comments_by_dd_authors.pkl')